In [ ]:
#fine tune model on med-easi dataset with flan-t5-base to get baseline model

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
import torch

#load Med-EASi dataset
ds = load_dataset("cbasu/Med-EASi")
print(ds)
print("Train sample:\n", ds["train"][0])

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1397 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/196 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 1397
    })
    validation: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 196
    })
    test: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 300
    })
})
Train sample:
 {'Expert': '75-90 % of the affected people have mild intellectual disability.', 'Simple': "People with syndromic intellectual disabi

In [ ]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(batch):
    inputs = batch["Expert"]
    targets = batch["Simple"]

    model_inputs = tokenizer(
        inputs,
        max_length=256,
        truncation=True,
    )

    labels = tokenizer(
        text_target=targets,
        max_length=256,
        truncation=True,
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

cols_to_remove = ds["train"].column_names

tokenized_ds = ds.map(
    preprocess,
    batched=True,
    remove_columns=cols_to_remove,
)

tokenized_ds

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/1397 [00:00<?, ? examples/s]

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1397
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 196
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Using device:", device)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Using device: cuda


In [ ]:
import wandb
wandb.init(project="Med-EASE", name="flan-t5-base-rl-baseline")

#using best args from experiment
training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoints/flan_t5_baseline",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=3e-5,
    num_train_epochs=5,
    warmup_steps=1000,
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
    load_best_model_at_end=True,
    tpu_num_cores=8,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

train_result = trainer.train()
best_ckpt = trainer.state.best_model_checkpoint
best_metric = trainer.state.best_metric

print("Best checkpoint path:", best_ckpt)
print("Best metric", best_metric)

eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▄▇█
train/global_step,▁▄▇█
train/grad_norm,▄▁█
train/learning_rate,▁▄█
train/loss,█▆▁
eval/loss,1.88492
eval/runtime,1.1505


/tmp/ipython-input-661495471.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,1.486900,1.862019
2,1.549200,1.821185
3,1.476000,1.794517
4,1.337900,1.778186
5,1.296900,1.762969


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


Best checkpoint path: checkpoints/flan_t5_baseline/checkpoint-875
Best metric 1.7629690170288086


In [ ]:
wandb.finish()

eval/loss,█▅▃▂▁
eval/runtime,▁▅▄█▄
eval/samples_per_second,█▄▅▁▅
eval/steps_per_second,█▄▅▁▅
train/epoch,▁▁▂▂▂▃▃▄▄▄▄▅▅▅▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▃▃▄▄▄▄▅▅▅▆▆▇▇▇▇███
train/grad_norm,▅▄▅▅▄▁▇▂▂▃▃▆▂█▂▁▃
train/learning_rate,▁▁▂▂▃▃▄▄▄▅▅▆▆▇▇██
train/loss,█▇▅▆▅▆▇▃▆▅▄▄▄▂▂▃▁
eval/loss,1.76297
eval/runtime,1.1348


In [ ]:
import math
import textstat
import ast

def readability_reward(text: str, min_grade: float = 5.0, max_grade: float = 15.0) -> float:
    """
    Return score between 0 and 1, where 1 = easiest to read, 0 = very hard
    map Flesch-Kincaid grade from [min_grade, max_grade] to [1,0].
    """
    grade = textstat.flesch_kincaid_grade(text)
    if math.isnan(grade):
        return 0.0

    grade = max(min_grade, min(max_grade, grade))
    score = 1.0 - (grade - min_grade) / (max_grade - min_grade)
    return score


def parse_expert_terms(s: str):
    """
    expert_terms is stored like "['intellectual disability']" within med-easi dataset.
    parse it into a list and convert to lowercase
    """
    if s is None:
        return []
    try:
        terms = ast.literal_eval(s)
        if isinstance(terms, list):
            return [t.lower() for t in terms]
    except Exception:
        pass
    return []

def entity_reward(prediction: str, expert_terms_str: str) -> float:
    """
    return fraction of expert terms present in the prediction output.
    """
    terms = parse_expert_terms(expert_terms_str)
    if not terms:
        return 1.0

    pred_lower = prediction.lower()
    present = sum(int(term in pred_lower) for term in terms)
    return present / len(terms)

def noncopy_reward(prediction: str, expert: str) -> float:
    """
    reward b/w 0 and 1, where 1 = very different from Expert field,
    0 = identical / heavily overlapping from Expert field.
    """
    pred_tokens = set(prediction.lower().split())
    expert_tokens = set(expert.lower().split())
    if not pred_tokens or not expert_tokens:
        return 0.0
    jaccard = len(pred_tokens & expert_tokens) / len(pred_tokens | expert_tokens) #use jaccard similarity
    #high overlap ==> low reward
    return 1.0 - jaccard

#favor outputs shorter than expert field
def length_reward(prediction: str, expert: str) -> float:
    """
    Reward b/w 0 and 1: 1 if prediction is much shorter than expert,
    decreasing as it approaches or exceeds expert length.
    """
    len_pred = len(prediction.split())
    len_expert = len(expert.split())
    if len_expert == 0:
        return 0.0

    ratio = len_pred / len_expert  #want this < 1 ideally
    if ratio >= 1.0:
        return 0.0
    #if ratio <= 0.5 -> ~1.0, if ratio approaching 1.0 -> 0.0
    return 1.0 - (ratio - 0.5) / 0.5 if ratio > 0.5 else 1.0


def combined_reward(prediction, expert, expert_terms_str, w_read=0.40, w_ent=0.15, w_noncopy=0.35, w_len=0.10):
    r_read = readability_reward(prediction)
    r_ent  = entity_reward(prediction, expert_terms_str)
    r_nc   = noncopy_reward(prediction, expert)
    r_len  = length_reward(prediction, expert)

    return (
        w_read * r_read +
        w_ent  * r_ent +
        w_noncopy * r_nc +
        w_len * r_len
    )



def generate_with_reranking(batch, num_candidates = 4):
    experts = batch["Expert"]
    expert_terms_list = batch["expert_terms"]

    preds_rl = []

    model.eval()
    for expert, expert_terms_str in zip(experts, expert_terms_list):
        inputs = tokenizer(
            expert,
            return_tensors="pt",
            truncation=True,
            max_length=256,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=256,
                do_sample=True,
                top_k=50,
                top_p=0.9,
                temperature=1.2,
                num_return_sequences=num_candidates,
            )

        candidates = tokenizer.batch_decode(outputs, skip_special_tokens=True)

        best_cand = None
        best_reward = -1.0
        for cand in candidates:
            r = combined_reward(
                prediction=cand,
                expert=expert,
                expert_terms_str=expert_terms_str,
            )
            if r > best_reward:
                best_reward = r
                best_cand = cand

        preds_rl.append(best_cand)

    batch["pred_rl"] = preds_rl
    return batch




In [ ]:
examples = [
    "The patient was diagnosed with congestive heart failure and exhibited pulmonary oedema requiring diuretics and oxygen supplementation.",
    "Administration of 5-fluorouracil can lead to myelosuppression and mucositis; dose adjustments may be necessary in elderly patients.",
    "Hyponatraemia in this context is likely due to SIADH caused by the small-cell lung carcinoma, necessitating fluid restriction and close monitoring of sodium levels.",
    "The MRI demonstrates a hyperintense lesion on T2-weighted images consistent with demyelination in the periventricular white matter.",
    "The patient presents with chronic kidney disease stage 3b, eGFR 35 mL/min/1.73m2, and requires medication dose review to avoid nephrotoxicity."
]

for i, text in enumerate(examples, 1):
    # generate_with_reranking expects a batch, so we create one for a single example
    single_example_batch = {
        "Expert": [text],
        "expert_terms": ["[]"] # No expert terms for these custom examples
    }

    # The function returns a dictionary with 'pred_rl'
    result = generate_with_reranking(single_example_batch)
    simplified_text = result["pred_rl"][0]

    # Print original and simplified outputs
    print(f"Example {i} - Original:\n{text}\n")
    print(f"Example {i} - Simplified:\n{simplified_text}\n")
    print("-" * 80)

Example 1 - Original:
The patient was diagnosed with congestive heart failure and exhibited pulmonary oedema requiring diuretics and oxygen supplementation.

Example 1 - Simplified:
This patient developed hyperthyroidism and is at increased risk for congestive heart failure.

--------------------------------------------------------------------------------
Example 2 - Original:
Administration of 5-fluorouracil can lead to myelosuppression and mucositis; dose adjustments may be necessary in elderly patients.

Example 2 - Simplified:
5-fluorouracil can lead to meelosuppression and mucositis. Some elderly people may benefit from taking this drug.

--------------------------------------------------------------------------------
Example 3 - Original:
Hyponatraemia in this context is likely due to SIADH caused by the small-cell lung carcinoma, necessitating fluid restriction and close monitoring of sodium levels.

Example 3 - Simplified:
This may lead to SIADH, resulting in very low blood sug